In [1]:
import numpy as np
import pandas as pd
import os
import re

# TODO

1. data, location, 없는 데이터 없애기
2. date 분리 - 년 월 일
3. location 분리 - 나라 도시(주)
4. time 분리 - 시 분
5. operator 분리 - military private 항공사 - unique 해보기
6. flight# - unique 해보기
7. route 분리 - 이거는 자바 스크립트가 어떻게 읽는지에 따라 달라지는데
8. aboard 분리 - passengers crew - ? 있는게 얼마나 있는지 찾기
9. fatalities - passengers crew - ? 있는게 얼마나 있는지 찾기

# Crawling TODO

1. Famous Death

In [2]:
path = "../../dataset/data.csv"
data = pd.read_csv(path, encoding="cp949")
data

,Date:,Time:,Location:,Operator:,Flight#:,Route:,ACType:,Registration:,cn/ln:,Aboard:,Fatalities:,Ground:,Summary:
0,17.Sep.08,1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2 _ (passengers:1_ crew:1),1 _ (passengers:1_ crew:0),0,"During a demonstration flight, a U.S. Army fly..."
1,7.Sep.09,?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1 _ (passengers:0_ crew:1),1 _ (passengers:0_ crew:0),0,Eugene Lefebvre was the first pilot to ever be...
2,12.Jul.12,630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5 _ (passengers:0_ crew:5),5 _ (passengers:0_ crew:5),0,First U.S. dirigible Akron exploded just offsh...
3,6.Aug.13,?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1 _ (passengers:0_ crew:1),1 _ (passengers:0_ crew:1),0,The first fatal airplane accident in Canada oc...
4,9.Sep.13,1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20 _ (passengers:?_ crew:?),14 _ (passengers:?_ crew:?),0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,15.Jan.23,1050,"Pokhara, Nepal",Yeti Airlines,YT691,Kathmandu - Pokhara,ATR 72-500-72-212-A,9NANC,754,72 _ (passengers:68_ crew:4),72 _ (passengers:68_ crew:4),0,"Before approach, the pilot requested a change ..."
5030,16.Sep.23,?,"Barcelos, Brazil",Manaus Aerotaxi,?,?,Embraer EMB-110P1 Bandeirante,PT-SOG,110490,14 _ (passengers:12_ crew:2),14 _ (passengers:12_ crew:2),0,The air taxi crashed in heavy rain while attem...
5031,29.Oct.23,630,"Rio Branco, Brazil",ART Taxi Aero,?,Rio Branco - Envira,Cessna 208B Grand Caravan,PT-MEE,208B0344,10 _ (passengers:12_ crew:2),10 _ (passengers:12_ crew:2),0,The air taxi crashed into a heavy wooded area ...
5032,1.Nov.23,1415,"Temixco, Morelos, Mexico",Jet Rescue Air Ambulance,?,Toluca - Cuernavaca Airport,Learjet 35A,XA-IRE,35A-354,4 _ (passengers:2_ crew:2),4 _ (passengers:2_ crew:2),0,"A business jet operating as an air ambulance, ..."


In [3]:
data = data[data["Location:"] != "?"] # 1
data.shape

(5029, 13)

In [4]:
date = pd.to_datetime(data["Date:"])
data.loc[:,"year"] = date.dt.year
data.loc[:,"month"] = date.dt.month
data.loc[:,"day"] = date.dt.day
data.shape

/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/1237329074.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date = pd.to_datetime(data["Date:"])
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/1237329074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"year"] = date.dt.year
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/1237329074.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

(5029, 16)

In [5]:
# API_KEY = """"""
# base_query = """
# What is the country, state and city name of following texts 

# [QUERY]

# If the text don't have city, state or country, answer "?".
# Answer only country, state and city like "Korea, ?, Busan"
# """
# batch_size = 20
# count = 0
# batch = []
# import openai
# client = openai.OpenAI(api_key=API_KEY)
# for i, loc in enumerate(data["Location:"]):
#     count += 1
#     loc = loc.split("\n")[0]
#     """Ver. single"""
#     query = base_query.replace("[QUERY]", loc)
#     completion = client.chat.completions.create(
#         # model="gpt-3.5-turbo",
#         model="gpt-4",
#         messages=[
#             {"role": "system", "content": "You answer country, state and city name of following text."},
#             {"role": "user", "content": query},
#         ],
#         n=1  
#     )
#     with open(f"../../dataset/instant/single_{count}.txt", "w+") as fd:
#         fd.write(f"{i+1}\n{loc}\n{completion.choices[0].message.content}")


#     """Ver. batch"""
#     # if len(batch) == batch_size:
#     #     count += 1
#     #     batch_loc = "\n".join(batch)
#     #     query = base_query.replace("[QUERY]", batch_loc)
#     #     completion = client.chat.completions.create(
#     #         # model="gpt-3.5-turbo",
#     #         model="gpt-4",
#     #         messages=[
#     #             {"role": "system", "content": "You answer country, state and city name of following text."},
#     #             {"role": "user", "content": query},
#     #         ],
#     #         n=1  
#     #     )
#     #     with open(f"../../dataset/instant/batch_{count}.txt", "w+") as fd:
#     #         fd.write(f"{loc}\n{completion.choices[0].message.content}")
#     #     batch.clear()
#     # else:
#     #     batch.append(loc)

In [6]:
loc_inspect_path = "../../dataset/instant"
country = []
state = []
city = []
files = os.listdir(loc_inspect_path)

for file in sorted(files):
    with open(os.path.join(loc_inspect_path, file), "r") as fd:
        fd.readline()
        fd.readline()
        line = fd.readline()
        splitted = [x.strip() for x in line.split(",")]
        if splitted[0] == "USA":
            splitted[0] = "United States"
        country.append(splitted[0])
        state.append(splitted[1])
        city.append(splitted[2])

data.loc[:,"country"] = country
data.loc[:,"state"] = state
data.loc[:,"city"] = city

/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/53272042.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"country"] = country
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/53272042.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"state"] = state
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/53272042.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [7]:
hours = []
minutes = []
for time in data["Time:"]:
    # if "Z" in time:
    #     print(time)
    if time == "?":
        hours.append("?")
        minutes.append("?")
    else:
        # print(time)
        if len(time) == 1:
            hours.append(time)
            minutes.append("0")
        elif len(time) == 2:
            hours.append("0")
            minutes.append(time)
        elif len(time) == 3:
            hours.append(time[0])
            minutes.append(time[1:])
        elif len(time) == 4:
            if ":" in time:
                hours.append(time[:time.index(":")])
                minutes.append(time[time.index(":")+1:])
            else:
                hours.append(time[:2])
                minutes.append(time[2:])
        elif len(time) == 5:
            if "Z" in time:
                hours.append(time[:2])
                minutes.append(time[2:4])
            else:
                hours.append(time[:2])
                minutes.append(time[3:])
        elif len(time) == 7:
            hours.append(time[2:4])
            minutes.append(time[5:])
        elif len(time) == 6:
            # print(time)
            if time.startswith("c"):
                hours.append(time[2])
                minutes.append(time[4:])
            else:
                hours.append(time[:2])
                minutes.append(time[3:5])
        else:
            print(time, len(time), "wow")
data["Time:"].shape, len(hours), len(minutes)
data.loc[:, "hour"] = hours
data.loc[:, "minute"] = minutes
# data.loc[:, "hour"] = data["Time:"].apply(lambda x: "?" if x == "?" else x[:-2])
# data.loc[:, "minute"] = data["Time:"].apply(lambda x: "?" if x == "?" else x[-2:])

/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/1067982287.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "hour"] = hours
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/1067982287.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "minute"] = minutes


In [8]:
#e.t.c.
data.loc[:, "operator"] = data["Operator:"]
data.loc[:, "flight"] = data["Flight#:"]
data.loc[:, "actype"] = data["ACType:"]
data.loc[:, "registration"] = data["Registration:"]
data.loc[:, "cn_ln"] = data["cn/ln:"]
data.loc[:, "ground"] = data["Ground:"]
data.loc[:, "summary"] = data["Summary:"]

/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/3957082870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "operator"] = data["Operator:"]
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/3957082870.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "flight"] = data["Flight#:"]
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/3957082870.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [9]:
routes = []
for route in data["Route:"]:
    try:
        if "-" in route:
            split_route = route.split("-")
            split_route = [x.strip() for x in split_route]
            split_route_str = ",".join(split_route)
            routes.append(split_route_str)
        else:
            # routes.append([])
            routes.append("")
    except:
        # routes.append([])
        routes.append("")
data.loc[:, "route"] = routes
data

/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2321243546.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "route"] = routes


,Date:,Time:,Location:,Operator:,Flight#:,Route:,ACType:,Registration:,cn/ln:,Aboard:,...,hour,minute,operator,flight,actype,registration,cn_ln,ground,summary,route
0,17.Sep.08,1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2 _ (passengers:1_ crew:1),...,17,18,Military - U.S. Army,?,Wright Flyer III,?,1,0,"During a demonstration flight, a U.S. Army fly...",
1,7.Sep.09,?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1 _ (passengers:0_ crew:1),...,?,?,?,?,Wright Byplane,SC1,?,0,Eugene Lefebvre was the first pilot to ever be...,
2,12.Jul.12,630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5 _ (passengers:0_ crew:5),...,6,30,Military - U.S. Navy,?,Dirigible,?,?,0,First U.S. dirigible Akron exploded just offsh...,
3,6.Aug.13,?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1 _ (passengers:0_ crew:1),...,?,?,Private,?,Curtiss seaplane,?,?,0,The first fatal airplane accident in Canada oc...,
4,9.Sep.13,1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20 _ (passengers:?_ crew:?),...,18,30,Military - German Navy,?,Zeppelin L-1 (airship),?,?,0,The airship flew into a thunderstorm and encou...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,15.Jan.23,1050,"Pokhara, Nepal",Yeti Airlines,YT691,Kathmandu - Pokhara,ATR 72-500-72-212-A,9NANC,754,72 _ (passengers:68_ crew:4),...,10,50,Yeti Airlines,YT691,ATR 72-500-72-212-A,9NANC,754,0,"Before approach, the pilot requested a change ...","Kathmandu,Pokhara"
5030,16.Sep.23,?,"Barcelos, Brazil",Manaus Aerotaxi,?,?,Embraer EMB-110P1 Bandeirante,PT-SOG,110490,14 _ (passengers:12_ crew:2),...,?,?,Manaus Aerotaxi,?,Embraer EMB-110P1 Bandeirante,PT-SOG,110490,0,The air taxi crashed in heavy rain while attem...,
5031,29.Oct.23,630,"Rio Branco, Brazil",ART Taxi Aero,?,Rio Branco - Envira,Cessna 208B Grand Caravan,PT-MEE,208B0344,10 _ (passengers:12_ crew:2),...,6,30,ART Taxi Aero,?,Cessna 208B Grand Caravan,PT-MEE,208B0344,0,The air taxi crashed into a heavy wooded area ...,"Rio Branco,Envira"
5032,1.Nov.23,1415,"Temixco, Morelos, Mexico",Jet Rescue Air Ambulance,?,Toluca - Cuernavaca Airport,Learjet 35A,XA-IRE,35A-354,4 _ (passengers:2_ crew:2),...,14,15,Jet Rescue Air Ambulance,?,Learjet 35A,XA-IRE,35A-354,0,"A business jet operating as an air ambulance, ...","Toluca,Cuernavaca Airport"


In [10]:
total_aboards = []
passenger_aboards = []
crew_aboards = []

for aboard in data["Aboard:"]:
    regex = re.compile(r"(?P<total>[\d\?]+)+.+(passengers:(?P<passenger>[\d\?]+).+crew:(?P<crew>[\d\?]+))")
    match = regex.search(aboard)
    # print(aboard, match.groupdict())
    total_aboards.append(match.group("total"))
    passenger_aboards.append(match.group("passenger"))
    crew_aboards.append(match.group("crew"))

data.loc[:, "totalAboard"] = total_aboards
data.loc[:, "passengerAboard"] = passenger_aboards
data.loc[:, "crewAboard"] = crew_aboards

/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2794591492.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "totalAboard"] = total_aboards
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2794591492.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "passengerAboard"] = passenger_aboards
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2794591492.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [11]:
total_fatalities = []
passenger_fatalities = []
crew_fatalities = []

for fatality in data["Fatalities:"]:
    regex = re.compile(r"(?P<total>[\d\?]+)+.+(passengers:(?P<passenger>[\d\?]+).+crew:(?P<crew>[\d\?]+))")
    match = regex.search(fatality)
    # print(aboard, match.groupdict())
    total_fatalities.append(match.group("total"))
    passenger_fatalities.append(match.group("passenger"))
    crew_fatalities.append(match.group("crew"))

data.loc[:, "totalFatality"] = total_fatalities
data.loc[:, "passengerFatality"] = passenger_fatalities
data.loc[:, "crewFatality"] = crew_fatalities

/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2331079775.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "totalFatality"] = total_fatalities
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2331079775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "passengerFatality"] = passenger_fatalities
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2331079775.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [12]:
# year, month, day, hour, minute, total_aboard, passenger_aboard, crew_aboard, ground, total_fatality, passenger_fatality, crew_fatality
columns_to_change_qm = ["year", "month", "day", "hour", "minute", "totalAboard", "passengerAboard", "crewAboard", "ground", "totalFatality", "passengerFatality", "crewFatality"]
for column in columns_to_change_qm:
    # data.loc[:, column] = data[column].apply(lambda x: -1 if x == "?" else int(x))
    data.loc[:, column] = data[column].apply(lambda x: 0 if x == "?" else int(x))
    print("complete column: ", column)
data = data.astype(dict(zip(columns_to_change_qm, [int]*len(columns_to_change_qm))))

complete column:  year
complete column:  month
complete column:  day
complete column:  hour
complete column:  minute
complete column:  totalAboard
complete column:  passengerAboard
complete column:  crewAboard
complete column:  ground
complete column:  totalFatality
complete column:  passengerFatality
complete column:  crewFatality


/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2300673967.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2008 2009 2012 ... 2023 2023 1978]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  data.loc[:, column] = data[column].apply(lambda x: 0 if x == "?" else int(x))
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2300673967.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 9  9  7 ... 10 11  8]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  data.loc[:, column] = data[column].apply(lambda x: 0 if x == "?" else int(x))
/var/folders/t0/sh6dv3z57hsg22_shjcvw5c40000gn/T/ipykernel_17546/2300673967.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[17 

In [13]:
processed_data = data.iloc[:,13:]

In [14]:
processed_data.to_csv("../../dataset/processed_data.csv", index=False)

In [15]:
processed_data

,year,month,day,country,state,city,hour,minute,operator,flight,...,cn_ln,ground,summary,route,totalAboard,passengerAboard,crewAboard,totalFatality,passengerFatality,crewFatality
0,2008,9,17,United States,Virginia,Fort Myer,17,18,Military - U.S. Army,?,...,1,0,"During a demonstration flight, a U.S. Army fly...",,2,1,1,1,1,0
1,2009,9,7,France,?,Juvisy-sur-Orge,0,0,?,?,...,?,0,Eugene Lefebvre was the first pilot to ever be...,,1,0,1,1,0,0
2,2012,7,12,United States,New Jersey,Atlantic City,6,30,Military - U.S. Navy,?,...,?,0,First U.S. dirigible Akron exploded just offsh...,,5,0,5,5,0,5
3,2013,8,6,Canada,British Columbia,Victoria,0,0,Private,?,...,?,0,The first fatal airplane accident in Canada oc...,,1,0,1,1,0,1
4,2013,9,9,?,?,?,18,30,Military - German Navy,?,...,?,0,The airship flew into a thunderstorm and encou...,,20,0,0,14,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,2023,1,15,Nepal,?,Pokhara,10,50,Yeti Airlines,YT691,...,754,0,"Before approach, the pilot requested a change ...","Kathmandu,Pokhara",72,68,4,72,68,4
5030,2023,9,16,Brazil,?,Barcelos,0,0,Manaus Aerotaxi,?,...,110490,0,The air taxi crashed in heavy rain while attem...,,14,12,2,14,12,2
5031,2023,10,29,Brazil,Acre,Rio Branco,6,30,ART Taxi Aero,?,...,208B0344,0,The air taxi crashed into a heavy wooded area ...,"Rio Branco,Envira",10,12,2,10,12,2
5032,2023,11,1,Mexico,Morelos,Temixco,14,15,Jet Rescue Air Ambulance,?,...,35A-354,0,"A business jet operating as an air ambulance, ...","Toluca,Cuernavaca Airport",4,2,2,4,2,2
